In [ ]:
import tensorflow as tf
import cv2
import csv
import os
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
from sklearn import preprocessing

In [ ]:
IMG_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 20

MAX_SEQ_LENGTH = 25
NUM_FEATURES = 55296

In [ ]:
def load_video(path, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    numberOfFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT);

    skip_number = int(numberOfFrames) // MAX_SEQ_LENGTH

    try:
        for i in range(0, int(numberOfFrames), skip_number):
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, resize)
            frames.append(np.array(frame) / 255)
            if len(frames) == MAX_SEQ_LENGTH:
                break

    finally:
        cap.release()

    return np.array(frames)

In [ ]:
def featureExtractor():
    feature_extractor = tf.keras.models.load_model("/content/drive/MyDrive/weights-14.hdf5");
    return Model(feature_extractor.input, feature_extractor.layers[-2].output)
feature_extractor = featureExtractor()

/usr/local/lib/python3.7/dist-packages/keras/layers/core.py:1045: UserWarning: tensorflow.python.keras.applications.inception_resnet_v2 is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


In [ ]:
def load_all_videos(tags, paths, test_train, root_dir):
#     print(tags)
    train_features = []
    train_labels = []

    test_features = []
    test_labels = []

    for i, path in enumerate(paths):
        final_path = os.path.join(root_dir, path)
        frames = load_video(final_path)
        frames = frames[None, ...]

        for frame in frames:
            prediction = feature_extractor.predict(frame)
            if (test_train[i] == 'train'):
                train_features.append(prediction)
                train_labels.append(tags[i])
            else:
                test_features.append(prediction)
                test_labels.append(tags[i])

    return np.array(train_features), np.array(train_labels), np.array(test_features), np.array(test_labels)

In [ ]:
tags = []
paths = []
test_train = []
with open("videogamesVideosDataset/data_file.csv") as f:
    reader = csv.reader(f, delimiter="\t")
    for i in reader:
        tags.append(i[0].split(',')[2])
        paths.append(i[0].split(',')[3])
        test_train.append(i[0].split(',')[0])

In [ ]:
train_features, train_labels, test_features, test_labels = load_all_videos(tags, paths, test_train, "videogamesVideosDataset/")

In [ ]:
def lstmModel():
    model = Sequential()
#     model.add(Embedding(25, 512))
    model.add(LSTM(1024, return_sequences=False,
                   input_shape=(MAX_SEQ_LENGTH, 55296),
                   dropout=0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(21, activation='softmax'))

    return model

In [ ]:
model = lstmModel()

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
checkpointer = ModelCheckpoint('/content/drive/MyDrive/weights-lstm.hdf5', monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
le.fit(test_labels)
test_labels = le.transform(test_labels)

In [ ]:
le.fit(train_labels)
train_labels = le.transform(train_labels)

In [ ]:
test_labels = tf.one_hot(test_labels, 21)
train_labels = tf.one_hot(train_labels, 21)

In [ ]:
model.fit(
    train_features,
    train_labels,
#     batch_size=BATCH_SIZE,
    validation_data = (test_features, test_labels),
    epochs = EPOCHS,
    callbacks=[checkpointer]
)

Epoch 1/20
15/15 [==============================] - 15s 840ms/step - loss: 1.5431 - accuracy: 0.6203 - val_loss: 0.3119 - val_accuracy: 0.9798

Epoch 00001: val_loss improved from inf to 0.31187, saving model to /content/drive/MyDrive/weights-lstm.hdf5
Epoch 2/20
15/15 [==============================] - 13s 827ms/step - loss: 0.2706 - accuracy: 0.9581 - val_loss: 0.1280 - val_accuracy: 0.9697

Epoch 00002: val_loss improved from 0.31187 to 0.12797, saving model to /content/drive/MyDrive/weights-lstm.hdf5
Epoch 3/20
15/15 [==============================] - 12s 779ms/step - loss: 0.1195 - accuracy: 0.9845 - val_loss: 0.0797 - val_accuracy: 0.9798

Epoch 00003: val_loss improved from 0.12797 to 0.07972, saving model to /content/drive/MyDrive/weights-lstm.hdf5
Epoch 4/20
15/15 [==============================] - 12s 776ms/step - loss: 0.0551 - accuracy: 0.9978 - val_loss: 0.0481 - val_accuracy: 1.0000

Epoch 00004: val_loss improved from 0.07972 to 0.04810, saving model to /content/drive/My

In [ ]:
class_labels = [
  'Apex',
  'Csgo',
  'Clash Royale',
  "Deadth By Daylight",
  'Dota2',
  "Escape From Tarkov",
  "Fifa21",
  "Fortnite",
  "Free fire",
  "GtaV",
  "Lol",
  "Minecraft",
  "Overwatch",
  "pubg",
  "rainbows",
  "rocket league",
  "rust",
  "sea of thieves",
  "valorant",
  "warzone",
  "wow",
]

In [ ]:
path = "/content/drive/MyDrive/warzone_2.mp4"

In [ ]:
frames = load_video(path)

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/weights-lstm.hdf5")

In [ ]:
features = []
frames = frames[None, ...]
for frame in frames:
  # print(frame.shape)
  cnn_prediction = feature_extractor.predict(frame)
  features.append(cnn_prediction)

In [ ]:
features = np.array(features)

In [ ]:
prediction = model.predict(features)

In [ ]:
prediction_array = np.array(prediction[0])
prediction_array = prediction_array.round(decimals=3)
prediction_array = prediction_array * 100
print(prediction_array)

top1Arg = prediction_array.argmax()
top_1_prediction = prediction_array[top1Arg]
top_1_label = class_labels[top1Arg]

print(f"{top_1_label}: {top_1_prediction}")

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0. 100.   0.]
warzone: 100.0
